<a href="https://colab.research.google.com/github/Abdullahaml1/chatgpt-detect/blob/main/Chatgpt_Detctor_Playing_arround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

device = torch.device('cpu')

tokenizer = AutoTokenizer.from_pretrained("Hello-SimpleAI/chatgpt-detector-roberta")

model = AutoModelForSequenceClassification.from_pretrained("Hello-SimpleAI/chatgpt-detector-roberta", torchscript=True)
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [3]:
text = "Chat Gpt is awesome!"

inputs = tokenizer(text, return_tensors="pt")

inputs.to(device)
model.to(device)

with torch.no_grad():
    outputs = model(**inputs)
  
outputs
# answer_start_index = outputs.start_logits.argmax()
# answer_end_index = outputs.end_logits.argmax()

# predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
# tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# # target is "nice puppet"
# target_start_index = torch.tensor([14])
# target_end_index = torch.tensor([15])

# outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
# loss = outputs.loss
# round(loss.item(), 2)

(tensor([[ 0.6700, -0.8931]]),)

# Loading Dataset

In [4]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.9 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names
dataset = load_dataset('Hello-SimpleAI/HC3', name='all', split='train')
dataset = dataset.with_format('torch')

Generating train split: 0 examples [00:00, ? examples/s]

Dataset hc3 downloaded and prepared to /root/.cache/huggingface/datasets/Hello-SimpleAI___hc3/all/1.1.0/5af5910f9f3fe7aace30e32ad4c1ab776ca08183d00e9b2a091308549f69f683. Subsequent calls will reuse this data.


In [12]:
import random
import numpy as np

val_split = 0.1
test_split = 0.1




def split(val_split, test_split, dataset_len):
    indcies = list(range(dataset_len))
    val_split = int(len(indcies) * val_split)
    test_split = int(len(indcies) * test_split)

    np.random.shuffle(indcies)

    val_indcies = indcies[:val_split]
    test_indcies = indcies[val_split:(val_split + test_split)]
    train_indcies = indcies[(val_split + test_split):]
    return train_indcies, val_indcies, test_indcies

x, y, z = split(0.1, 0.1, len(dataset))
print(len(dataset))
print(len(x) + len(y) + len(z))

# Creating PT data samplers and loaders:
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)

24322
24322


In [ ]:
print('Length of data', len(dataset))
item = next(iter(dataset))
print(type(item))
print(item.keys())
print(item['source'])
print('Lenght of human answer:', len(item['human_answers']))
print('Lenght of chatgpt answer:', len(item['chatgpt_answers']))
print('type of human answer:', type(item['human_answers']))
print('question: ', item['question'])
print()
item['chatgpt_answers'][0]
item['human_answers'][0]

Length of data 24322
<class 'dict'>
dict_keys(['id', 'question', 'human_answers', 'chatgpt_answers', 'source'])
reddit_eli5
Lenght of human answer: 3
Lenght of chatgpt answer: 1
type of human answer: <class 'list'>
question:  Why is every book I hear about a " NY Times # 1 Best Seller " ? ELI5 : Why is every book I hear about a " NY Times # 1 Best Seller " ? Should n't there only be one " # 1 " best seller ? Please explain like I'm five.



'Basically there are many categories of " Best Seller " . Replace " Best Seller " by something like " Oscars " and every " best seller " book is basically an " oscar - winning " book . May not have won the " Best film " , but even if you won the best director or best script , you \'re still an " oscar - winning " film . Same thing for best sellers . Also , IIRC the rankings change every week or something like that . Some you might not be best seller one week , but you may be the next week . I guess even if you do n\'t stay there for long , you still achieved the status . Hence , # 1 best seller .'

In [ ]:
# preparing dataset

def class_labels(classes_l):
    '''
   :param classes_l: list of class names, numbers, ...
   :return class2ind(dict), ind2class(list)
   '''
    class2ind = {}
    ind2class = []
    for i in range(len(classes_l)):
        assert classes_l[i] not in class2ind, f'{classes_l[i]} is doblicated' 
        class2ind[classes_l[i]]= i
        ind2class.append(classes_l[i])

    return class2ind, ind2class

class2ind, ind2class = class_labels(['human_answers', 'chatgpt_answers'])
class2ind

{'human_answers': 0, 'chatgpt_answers': 1}

In [ ]:
# dataset
class UnpackedDataset(torch.utils.data.Dataset):
      """
      Pytorch data class for questions
      """
  
      def __init__(self, dataset, class2ind):
            self.samples = []

            for item in dataset:
                for c in class2ind:
                    if len(item[c]) >0:
                        s = {
                            'id': item['id'],
                            'question': item['question'],
                            'source': item['source'],
                            'answer': item[c][0],
                            'label': class2ind[c]
                        }
                        self.samples.append(s)
          
          
      ###You don't need to change this funtion
      def __getitem__(self, index):
          return self.samples[index]
            
      
      ###You don't need to change this funtion
      def __len__(self):
          return len(self.samples)

unpacked_ds = UnpackedDataset(dataset, class2ind)
print(len(unpacked_ds))

for i, item in enumerate(unpacked_ds):
    print('id:', item['id'])
    print('q:', item['question'])
    print('ans', item['answer'])
    print('s', item['source'])
    print('label', item['label'])
    print()
    if i > 5:
        break


48189
id: 0
q: Why is every book I hear about a " NY Times # 1 Best Seller " ? ELI5 : Why is every book I hear about a " NY Times # 1 Best Seller " ? Should n't there only be one " # 1 " best seller ? Please explain like I'm five.
ans Basically there are many categories of " Best Seller " . Replace " Best Seller " by something like " Oscars " and every " best seller " book is basically an " oscar - winning " book . May not have won the " Best film " , but even if you won the best director or best script , you 're still an " oscar - winning " film . Same thing for best sellers . Also , IIRC the rankings change every week or something like that . Some you might not be best seller one week , but you may be the next week . I guess even if you do n't stay there for long , you still achieved the status . Hence , # 1 best seller .
s reddit_eli5
label 0

id: 0
q: Why is every book I hear about a " NY Times # 1 Best Seller " ? ELI5 : Why is every book I hear about a " NY Times # 1 Best Seller "

In [ ]:
# batch function
class Batchify(object):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        """
        :param batch (batch)
        """
        ids = []
        questions = []
        sources = []
        labels = []
        ans = []
        for item in batch:
            ids.append( item['id'])
            questions.append(item['question'])
            sources.append(item['source'])
            ans.append(item['answer'])
            labels.append(item['label'])

        ans_tokens = self.tokenizer(ans, padding='max_length',
                                    truncation=True,
                                    return_tensors="pt")
        labels_ten = torch.tensor(labels, dtype=torch.long)

        return{'ans_tokens': ans_tokens,
               'questions': questions,
               'sources': sources,
               'labels': labels_ten}

test_sampler = torch.utils.data.sampler.SequentialSampler(unpacked_ds)
test_loader = torch.utils.data.DataLoader(unpacked_ds,
                                        batch_size=32,
                                        sampler=test_sampler,
                                        num_workers=2,
                                        collate_fn=Batchify(tokenizer))


NameError: ignored

In [ ]:
item = next(iter(test_loader))
print(item['labels'].shape)
model.to(device)
ans = item['ans_tokens']
ans['input_ids'].shape
ans['attention_mask'].shape
ans.to(device)
model(**ans).shape


torch.Size([32])


In [ ]:
# evalute funciton
def evaluate(data_loader, model, device):
    """
    evaluate the current model, get the accuracy for dev/test set
    Keyword arguments:
    data_loader: pytorch build-in data loader output
    model: model to be evaluated
    device: cpu of gpu
    """
                  
    model.to(device)
    model.eval()
    num_examples = 0
    error = 0                              
   
    total_loss = 0.0
    num_examples = 0
    with torch.no_grad():
        for idx, batch in enumerate(data_loader):
            ans_tokens = batch['ans_tokens']
            ans_tokens.to(device)
            labels = batch['labels']
      
              
            logits = model(**ans_tokens) # shape [batch x num_classes]
            top_n, top_i = logits.topk(1)
            num_examples += ans_tokens.size(0)
            error += torch.nonzero(top_i.squeeze() - torch.LongTensor(labels)).size(0)
   
   
        # Accuracy
        accuracy = 1 - error / num_examples
        avg_loss = total_loss/num_examples
    # print(f'Dev accuracy={accuracy:f}, Dev average Loss={avg_loss:f}')
    return accuracy


evaluate(test_loader, model, device)

NameError: ignored

In [ ]:
txt = "he Sun is the star at the center of the Solar System. It is a nearly perfect ball of hot plasma,[18][19] heated to incandescence by nuclear fusion reactions in its core. The Sun radiates this energy mainly as light, ultraviolet, and infrared radiation, and is the most important source of energy for life on Earth. The Sun's radius is about 695,000 kilometers (432,000 miles), or 109 times that of Earth. Its mass is about 330,000 times that of Earth, comprising about 99.86% of the total mass of the Solar System.[20] Roughly three-quarters of the Sun's mass consists of hydrogen (~73%); the rest is mostly helium (~25%), with much smaller quantities of heavier elements, including oxygen, carbon, neon, and iron.[21] The Sun is a G-type main-sequence star (G2V), informally called a yellow dwarf, though its light is actually white. It formed approximately 4.6 billion[a][14][22] years ago from the gravitational collapse of matter within a region of a large molecular cloud. Most of this matter gathered in the center, whereas the rest flattened into an orbiting disk that became the Solar System. The central mass became so hot and dense that it eventually initiated nuclear fusion in its core. It is thought that almost all stars form by this process. Every second, the Sun's core fuses about 600 million tons of hydrogen into helium, and in the process converts 4 million tons of matter into energy. This energy, which can take between 10,000 and 170,000 years to escape the core, is the source of the Sun's light and heat. When hydrogen fusion in its core has diminished to the point at which the Sun is no longer in hydrostatic equilibrium, its core will undergo a marked increase in density and temperature while its outer layers expand, eventually transforming the Sun into a red giant. It is calculated that the Sun will become sufficiently large to engulf the current orbits of Mercury and Venus, and render Earth uninhabitable in five billion years. After this, it will shed its outer layers and become a dense type of cooling star known as a white dwarf, and no longer produce energy by fusion, but still glow and give off heat from its previous fusion."
out = tokenizer(txt, padding='max_length', truncation=True, return_tensors='pt')
out['input_ids'].shape

torch.Size([1, 512])